In [1]:
x=7*8
x

56

### Getting a WMS to work in an ipyleaflet Map

The **ipyleaflet** Jupyter notebook extension must be enabled. The command from a Python cell looks like this... 

```
print('ok go')
!jupyter nbextension enable --py --sys-prefix ipyleaflet
```

**inspect** means that we right-click on the map control, select *inspect*, this opens at the right side of the
browser window, we select the *Network* tab, and we bump the control in some way to generate some tile requests. 
When these fail they appear in red ink. They may also be *pending* for some time. The various calls can be hovered 
or clicked to see in entirety. If they are working then compare them to what is happening in a failing map control;
where the difference in the call means the difference between working and not working. 


* [Marine Geoscience Data System](http://www.marine-geo.org/tools/web_services.php) gets an A+ with GMRT topography and clear links to GetCapabilities 
  * [Example](https://www.gmrt.org/services/mapserver/wms_merc?request=GetCapabilities&service=WMS&version=1.0.0)

```
wms = WMSLayer(url='https://www.gmrt.org/services/mapserver/wms_merc', layers='GMRT')
```

* [NASA Earth Observations](https://neo.sci.gsfc.nasa.gov/about/)
* [MARACOOS](http://wms.maracoos.org/wms/index) introduced the *styles* issue
  * Used the **inspect** browser feature to start figuring out: Why no data? 
  * But real progress was from **inspecting** the network calls from the [example leaflet map](http://wms.maracoos.org/wms/demo)
    * This got us from using the GetCap Style Name string to using a workable Style Name string

```
styles='contourf_average_jet_0.0_0.1_grid_False'
```

  * Here we find that *countourf_average...* does **not** occur in the GetCapabilities XML

* [NOAA NowCOAST](https://nowcoast.noaa.gov/help/#!section=wms-layer-ids) works for SST...
  * Get this: The SST layer for the WMS is named '1'. Like the number 1. I guess it is layer 1. 
  * Anyway it worked without further forensics so they get a B- for that
  
* [Federal Geographic Data Committee (FGDC.gov)](https://www.fgdc.gov/index_html#)
* [Integrated Marine Observing System (IMOS)](https://catalogue-imos.aodn.org.au/geonetwork/srv/eng/main.home)
  
### Further goals

* [time-enabled services like NOAA says](https://nowcoast.noaa.gov/help/#!section=time-enabled-map-services)
* Chlorophyll, MSLA, sea state, precip, wind speed, wind direction would all be good to have
* Tie in to specific lat/lon
* DN extraction
* Greyscale version re-colored
* NASA GIBS 

**stopper**

In [2]:
!jupyter nbextension enable --py --sys-prefix ipyleaflet

Enabling notebook extension jupyter-leaflet/extension...
      - Validating: OK


In [3]:
# from ipyleaflet import (Map, Marker, TileLayer, ImageOverlay, Polyline, Polygon, Rectangle, Circle, CircleMarker,
#     GeoJSON, DrawControl, WMSLayer)

from ipyleaflet import *

from traitlets import link

center, zoom, widString, hgtString = [46, -126], 7, '970px', '250px'
m = Map(center=center, zoom=zoom, layout=dict(width=widString, height=hgtString))
m2 = Map(center=center, zoom=zoom, layout=dict(width=widString, height=hgtString))

# print(m.zoom)
wms = WMSLayer(url='https://www.gmrt.org/services/mapserver/wms_merc', layers='GMRT')
# works; mid-Atlantic only
# wms2 = WMSLayer(url='http://wms.maracoos.org/wms/MODIS_Eight_Agg', \
#                 layers='mass_concentration_of_chlorophyll_in_sea_water', \
#                 styles='contourf_average_jet_0.0_0.1_grid_False')
# works but mid-Atlantic only
# wms2 = WMSLayer(url='http://wms.maracoos.org/wms/MURSST_Agg', \
#                 layers='sea_surface_temperature', \
#                 styles='contourf_average_jet_0.0_0.1_grid_False')
# doesn't work...
# wms2 = WMSLayer(url='https://neo.sci.gsfc.nasa.gov/wms', layers='AVHRR_CLIM_W', styles='rgb')
# SST but tiles unreliable
wms2 = WMSLayer(url='https://nowcoast.noaa.gov/arcgis/services/nowcoast/analysis_ocean_sfc_sst_time/MapServer/WmsServer',\
                layers='1')
# wms2 = WMSLayer(url='https://nowcoast.noaa.gov/arcgis/services/nowcoast/obs_meteocean_insitu_sfc_time/MapServer/WMSServer',\
#                  layers='34')
# Does not work (monthly)
# wms2 = WMSLayer(url='https://neo.sci.gsfc.nasa.gov/wms/wms', layers='MY1DMM_CHLORA', styles='rgb')
# Trying weekly, 'rgb' and 'gs' should both work but neither
# wms2 = WMSLayer(url='https://neo.sci.gsfc.nasa.gov/wms/wms', layers='MY1DMW_CHLORA', styles='gs')
# wms2 = WMSLayer(url='https://neo.sci.gsfc.nasa.gov/wms', layers='MY1DMW_CHLORA', styles='gs')
# wms2 = WMSLayer(url='https://neo.sci.gsfc.nasa.gov/wms/wms', layers='MODAL2_M_CLD_FR', styles='rgb')
# wms2 = WMSLayer(url='', layers='', styles='')

# change map size with ..., layout=dict(width='600px', height='400px'))
m.add_layer(wms)
m2.add_layer(wms2)

dc = DrawControl()
def handle_draw(self, action, geo_json):
    print(action)
    print(geo_json)
dc.on_draw(handle_draw)

m.add_control(dc)
map_center_link = link((m, 'center'), (m2, 'center'))
map_zoom_link = link((m, 'zoom'), (m2, 'zoom'))

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [4]:
m2

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …